_Lambda School Data Science - Model Validation_

## Example solution to the Cross-Validation assignment — plus Feature Selection!

See also Sebastian Raschka's example, [Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb).

In [1]:
# We'll modify a project from Python Data Science Handbook by Jake VanderPlas
# https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html#Example:-Predicting-Bicycle-Traffic
    
# Predicting Bicycle Traffic

# As an example, let's take a look at whether we can predict the number of 
# bicycle trips across Seattle's Fremont Bridge based on weather, season, 
# and other factors.

# We will join the bike data with another dataset, and try to determine the 
# extent to which weather and seasonal factors—temperature, precipitation, 
# and daylight hours—affect the volume of bicycle traffic through this corridor. 
# Fortunately, the NOAA makes available their daily weather station data 
# (I used station ID USW00024233) and we can easily use Pandas to join 
# the two data sources.


import numpy as np
import pandas as pd
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler


def load(): 
    fremont_bridge = 'https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD'
    
    bicycle_weather = 'https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/BicycleWeather.csv'

    counts = pd.read_csv(fremont_bridge, index_col='Date', parse_dates=True, 
                         infer_datetime_format=True)

    weather = pd.read_csv(bicycle_weather, index_col='DATE', parse_dates=True, 
                          infer_datetime_format=True)

    daily = counts.resample('d').sum()
    daily['Total'] = daily.sum(axis=1)
    daily = daily[['Total']] # remove other columns

    weather_columns = ['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'AWND']
    daily = daily.join(weather[weather_columns], how='inner')
    
    # Make a feature for yesterday's total
    daily['Total_yesterday'] = daily.Total.shift(1)
    daily = daily.drop(index=daily.index[0])
    
    return daily

    
def split(daily):
    # Hold out an "out-of-time" test set, from the last 100 days of data
    
    train = daily[:-100]
    test = daily[-100:]
    
    X_train = train.drop(columns='Total')
    y_train = train.Total

    X_test  = test.drop(columns='Total')
    y_test  = test.Total
    
    return X_train, X_test, y_train, y_test


def jake_wrangle(X):  
    X = X.copy()

    # patterns of use generally vary from day to day; 
    # let's add binary columns that indicate the day of the week:
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for i, day in enumerate(days):
        X[day] = (X.index.dayofweek == i).astype(float)


    # we might expect riders to behave differently on holidays; 
    # let's add an indicator of this as well:
    from pandas.tseries.holiday import USFederalHolidayCalendar
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays('2012', '2016')
    X = X.join(pd.Series(1, index=holidays, name='holiday'))
    X['holiday'].fillna(0, inplace=True)


    # We also might suspect that the hours of daylight would affect 
    # how many people ride; let's use the standard astronomical calculation 
    # to add this information:
    def hours_of_daylight(date, axis=23.44, latitude=47.61):
        """Compute the hours of daylight for the given date"""
        days = (date - pd.datetime(2000, 12, 21)).days
        m = (1. - np.tan(np.radians(latitude))
             * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
        return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

    X['daylight_hrs'] = list(map(hours_of_daylight, X.index))


    # temperatures are in 1/10 deg C; convert to C
    X['TMIN'] /= 10
    X['TMAX'] /= 10

    # We can also calcuate the average temperature.
    X['Temp (C)'] = 0.5 * (X['TMIN'] + X['TMAX'])


    # precip is in 1/10 mm; convert to inches
    X['PRCP'] /= 254

    # In addition to the inches of precipitation, let's add a flag that 
    # indicates whether a day is dry (has zero precipitation):
    X['dry day'] = (X['PRCP'] == 0).astype(int)


    # Let's add a counter that increases from day 1, and measures how many 
    # years have passed. This will let us measure any observed annual increase 
    # or decrease in daily crossings:
    X['annual'] = (X.index - X.index[0]).days / 365.

    return X


def wrangle(X):
    # From Daniel H (DS1 KotH)
    X = X.copy()
    X = X.replace(-9999, 0)
    X = jake_wrangle(X)
    
    X['PRCP_yest'] = X.PRCP.shift(1).fillna(X.PRCP.mean())
    X['Windchill'] = (((X['Temp (C)'] * (9/5) + 32) * .6215) + 34.74) - (35.75 * (X['AWND']** .16)) + (.4275 * (X['Temp (C)'])) * (X['AWND'] ** .16)
    X['Rl_Cold'] = (((X['Temp (C)'] * (9/5) + 32) - X['Windchill']) -32) * (5/9)
    X['TMIN_ln'] = X['TMIN'] **2
    
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for i, month in enumerate(months):
        X[month] = (X.index.month == i+1).astype(float)
    
    return X

In [2]:
# Download and join data into a dataframe
data = load()

In [3]:
%%time

# Split data into train and test
X_train, X_test, y_train, y_test = split(data)

# Do the same wrangling to X_train and X_test
X_train = wrangle(X_train)
X_test  = wrangle(X_test)

# Define an estimator and param_grid
pipe = make_pipeline(
    RobustScaler(), 
    SelectKBest(f_regression), 
    Ridge())

param_grid = {
    'selectkbest__k': range(1, len(X_train.columns)), 
    'ridge__alpha': [0.1, 1.0, 10.]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='neg_mean_absolute_error', 
                  verbose=1)

gs.fit(X_train, y_train)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', -validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

Fitting 3 folds for each of 102 candidates, totalling 306 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Cross-Validation Score: 297.1414603084522

Best estimator: Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=30, score_func=<function f_regression at 0x1a1b2dbc80>)), ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

CPU times: user 15.2 s, sys: 67.4 ms, total: 15.3 s
Wall time: 3.96 s


[Parallel(n_jobs=1)]: Done 306 out of 306 | elapsed:    3.8s finished


In [4]:
# Predict with X_test features
y_pred = gs.predict(X_test)

# Compare predictions to y_test labels
test_score = mean_absolute_error(y_test, y_pred)
print('Test Score:', test_score)

Test Score: 321.9835901148292


In [5]:
# Or use the grid search's score method, 
# which combines these steps
test_score = gs.score(X_test, y_test)
print('Test Score:', -test_score)

Test Score: 321.9835901148292


In [6]:
# Which features were selected?
selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
PRCP
TMAX
TMIN
AWND
Total_yesterday
Mon
Tue
Wed
Thu
Sat
Sun
holiday
daylight_hrs
Temp (C)
dry day
annual
PRCP_yest
Windchill
Rl_Cold
TMIN_ln
Jan
Feb
Mar
May
Jun
Jul
Aug
Sep
Nov
Dec

Features not selected:
SNOW
SNWD
Fri
Apr
Oct


## BONUS: Recursive Feature Elimination!

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html

In [7]:
from sklearn.feature_selection import RFECV

X_train_scaled = RobustScaler().fit_transform(X_train)
rfe = RFECV(Ridge(alpha=1.0), scoring='neg_mean_absolute_error', cv=3)
X_train_subset = rfe.fit_transform(X_train_scaled, y_train)

all_names = X_train.columns
selected_mask = rfe.support_
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
PRCP
TMAX
TMIN
AWND
Total_yesterday
Mon
Tue
Wed
Thu
Fri
Sat
Sun
holiday
daylight_hrs
Temp (C)
dry day
Windchill
Rl_Cold
TMIN_ln
Mar
May
Jun
Oct
Dec

Features not selected:
SNOW
SNWD
annual
PRCP_yest
Jan
Feb
Apr
Jul
Aug
Sep
Nov


In [8]:
X_train_subset = pd.DataFrame(X_train_subset, columns=selected_names)

In [9]:
X_test_subset = rfe.transform(X_test)
X_test_subset = pd.DataFrame(X_test_subset, columns=selected_names)

In [10]:
print(X_train.shape, X_train_subset.shape, X_test.shape, X_test_subset.shape)

(963, 35) (963, 24) (100, 35) (100, 24)


# RFE again, but with polynomial features and interaction terms!

In [11]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_train_polynomial = poly.fit_transform(X_train)

print(X_train.shape, X_train_polynomial.shape)

(963, 35) (963, 666)


In [12]:
from sklearn.feature_selection import RFECV

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train_polynomial)

rfe = RFECV(Ridge(alpha=1.0), scoring='neg_mean_absolute_error', 
            step=10, cv=3, verbose=1)

X_train_subset = rfe.fit_transform(X_train_scaled, y_train)

Fitting estimator with 666 features.
Fitting estimator with 656 features.
Fitting estimator with 646 features.
Fitting estimator with 636 features.
Fitting estimator with 626 features.
Fitting estimator with 616 features.
Fitting estimator with 606 features.
Fitting estimator with 596 features.
Fitting estimator with 586 features.
Fitting estimator with 576 features.
Fitting estimator with 566 features.
Fitting estimator with 556 features.
Fitting estimator with 546 features.
Fitting estimator with 536 features.
Fitting estimator with 526 features.
Fitting estimator with 516 features.
Fitting estimator with 506 features.
Fitting estimator with 496 features.
Fitting estimator with 486 features.
Fitting estimator with 476 features.
Fitting estimator with 466 features.
Fitting estimator with 456 features.
Fitting estimator with 446 features.
Fitting estimator with 436 features.
Fitting estimator with 426 features.
Fitting estimator with 416 features.
Fitting estimator with 406 features.
F

Fitting estimator with 246 features.
Fitting estimator with 236 features.
Fitting estimator with 226 features.
Fitting estimator with 216 features.
Fitting estimator with 206 features.
Fitting estimator with 196 features.
Fitting estimator with 186 features.
Fitting estimator with 176 features.
Fitting estimator with 166 features.


In [13]:
all_names = poly.get_feature_names(X_train.columns)
selected_mask = rfe.support_
selected_names = [name for name, selected in zip(all_names, selected_mask) if selected]

print(f'{rfe.n_features_} Features selected:')
for name in selected_names:
    print(name)

156 Features selected:
PRCP
TMAX
Sun
holiday
daylight_hrs
Temp (C)
annual
Mar
May
Nov
PRCP Tue
PRCP Fri
PRCP Sat
PRCP Sun
PRCP holiday
PRCP daylight_hrs
PRCP Jan
PRCP Feb
PRCP Mar
PRCP Apr
PRCP May
PRCP Jun
PRCP Jul
PRCP Aug
PRCP Sep
PRCP Nov
PRCP Dec
TMAX^2
TMAX TMIN
TMAX Rl_Cold
TMIN AWND
TMIN Total_yesterday
TMIN Windchill
AWND^2
AWND Total_yesterday
AWND daylight_hrs
AWND Temp (C)
AWND dry day
AWND annual
AWND Windchill
AWND Rl_Cold
AWND TMIN_ln
Total_yesterday^2
Total_yesterday daylight_hrs
Total_yesterday dry day
Total_yesterday annual
Total_yesterday Rl_Cold
Mon holiday
Mon daylight_hrs
Mon annual
Mon Jan
Mon Feb
Mon Mar
Mon May
Mon Jun
Mon Oct
Mon Nov
Tue daylight_hrs
Tue dry day
Tue annual
Tue Mar
Tue Apr
Tue May
Tue Aug
Tue Sep
Tue Dec
Wed holiday
Wed daylight_hrs
Wed PRCP_yest
Wed Jan
Wed Mar
Wed Apr
Wed Sep
Wed Oct
Thu holiday
Thu daylight_hrs
Thu dry day
Thu annual
Thu PRCP_yest
Thu Jan
Thu Feb
Thu Mar
Thu Apr
Thu May
Thu Jun
Thu Sep
Thu Oct
Thu Nov
Thu Dec
Fri daylight_hr

In [14]:
# Define an estimator and param_grid

ridge = Ridge()

param_grid = {
    'alpha': [0.1, 1.0, 10.]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(ridge, param_grid=param_grid, cv=3, 
                  scoring='neg_mean_absolute_error', 
                  verbose=1)

gs.fit(X_train_subset, y_train)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', -validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

Fitting 3 folds for each of 3 candidates, totalling 9 fits

Cross-Validation Score: 247.80985367683869

Best estimator: Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s finished


In [15]:
# Do the same transformations to X_test
X_test_polynomial = poly.transform(X_test)
X_test_scaled = scaler.transform(X_test_polynomial)
X_test_subset = rfe.transform(X_test_scaled)

# Use the grid search's score method with X_test_subset
test_score = gs.score(X_test_subset, y_test)
print('Test Score:', -test_score)

Test Score: 328.5706677677046


In [16]:
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_features.csv
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_labels.csv
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/test_features.csv
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/sample_submission.csv

--2019-03-07 19:46:11--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_features.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3833210 (3.7M) [text/plain]
Saving to: ‘train_features.csv.4’

train_features.csv. 100%[===================>]   3.66M  1.64MB/s    in 2.2s    

2019-03-07 19:46:14 (1.64 MB/s) - ‘train_features.csv.4’ saved [3833210/3833210]

--2019-03-07 19:46:14--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Sprint-4-Model-Validation/master/module-1-begin-modeling-process/bank-marketing/train_labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.196.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.196.133|:443..

In [17]:
%matplotlib inline
import warnings
import category_encoders as ce
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

X_train = pd.read_csv('train_features.csv').drop(columns='id')
y_train = pd.read_csv('train_labels.csv')['y']
X_test = pd.read_csv('test_features.csv').drop(columns='id')
sample_submission = pd.read_csv('sample_submission.csv')

X_train.shape, y_train.shape, X_test.shape, sample_submission.shape

((30891, 19), (30891,), (10297, 19), (10297, 2))

In [18]:
train_features = pd.read_csv('train_features.csv').drop(columns='id')
test_features = pd.read_csv('test_features.csv').drop(columns='id')
y_train = pd.read_csv('train_labels.csv')['y']
sample_submission = pd.read_csv('sample_submission.csv')

In [19]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

Xy = train_features.select_dtypes(include=['float64','int'])
X = StandardScaler().fit_transform(Xy)
X = pd.DataFrame(X, columns=Xy.columns)

print(Xy.columns)

vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')
[1.0180143929184464, 1.0331454608367934, 1.630987920721132, 1.8116280448816775, 33.143383657897864, 6.3579671852812405, 2.650108554330848, 64.72591256034646, 31.726059799279078]


/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [20]:
#drop because >10 vif
train_features.drop(columns=['emp.var.rate','euribor3m','nr.employed'],inplace=True)
test_features.drop(columns=['emp.var.rate','euribor3m','nr.employed'],inplace=True)

In [21]:
train_features['job'].replace(['services','housemaid'],'all_services',inplace=True)
test_features['job'].replace(['services','housemaid'],'all_services',inplace=True)
train_features['job'].replace(['unemployed','retired'],'no_employment',inplace=True)
test_features['job'].replace(['unemployed','retired'],'no_employment',inplace=True)
train_features['job'].replace(['management','blue-collar','admin.'],'corporate',inplace=True)
test_features['job'].replace(['management','blue-collar','admin.'],'corporate',inplace=True)
train_features['job'].replace(['self-employed','entrepreneur'],'self-employed',inplace=True)
test_features['job'].replace(['self-employed','entrepreneur'],'self-employed',inplace=True)
train_features['marital'].replace(['single','divorced'],'single',inplace=True)
test_features['marital'].replace(['single','divorced'],'single',inplace=True)
train_features['ln_campaign'] = np.log(train_features['campaign'])
test_features['ln_campaign'] = np.log(test_features['campaign'])


train_features.drop(columns=['campaign'],inplace=True)
test_features.drop(columns=['campaign'],inplace=True)


In [22]:
train_features.columns == test_features.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [23]:
train_features.shape,test_features.shape

((30891, 16), (10297, 16))

In [24]:
ohe = ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore',impute_missing=False)
train_features_encoded = ohe.fit_transform(train_features)
test_features_encoded = ohe.fit_transform(test_features)
train_features_encoded.shape, test_features_encoded.shape


((30891, 53), (10297, 52))

In [25]:
columns = train_features_encoded.columns
test_cols = test_features_encoded.columns
for col in columns:
    if col not in test_cols:
        print(col)

default_yes


In [26]:
train_features_encoded.drop(columns='default_yes',inplace=True)

In [27]:
test_features['default'].unique()

array(['no', 'unknown'], dtype=object)

In [28]:
len(train_features_encoded.columns) == len(test_features_encoded.columns)

True

In [29]:
model = LogisticRegression()
model.fit(train_features_encoded, y_train)
print(train_features_encoded.columns)
model.coef_

/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Index(['job_all_services', 'job_corporate', 'job_self-employed',
       'job_technician', 'job_no_employment', 'job_student', 'job_unknown',
       'marital_single', 'marital_married', 'marital_unknown',
       'education_high.school', 'education_university.degree',
       'education_unknown', 'education_basic.4y',
       'education_professional.course', 'education_basic.9y',
       'education_basic.6y', 'education_illiterate', 'default_no',
       'default_unknown', 'housing_yes', 'housing_no', 'housing_unknown',
       'loan_yes', 'loan_no', 'loan_unknown', 'contact_cellular',
       'contact_telephone', 'month_may', 'month_nov', 'month_jul', 'month_dec',
       'month_apr', 'month_mar', 'month_sep', 'month_aug', 'month_jun',
       'month_oct', 'day_of_week_thu', 'day_of_week_tue', 'day_of_week_mon',
       'day_of_week_wed', 'day_of_week_fri', 'poutcome_nonexistent',
       'poutcome_failure', 'poutcome_success', 'age', 'pdays', 'previous',
       'cons.price.idx', 'cons.conf.idx',

array([[-2.33430820e-01, -1.36124575e-01, -1.85853198e-01,
        -1.52733203e-01,  2.66893476e-01,  4.74015689e-01,
        -3.34195646e-03,  5.20903575e-02, -5.94704574e-02,
         3.68055112e-02,  2.04079113e-02,  1.35015926e-01,
         1.41755110e-01, -7.40894174e-02, -4.43085129e-02,
        -1.85208337e-01,  9.35132366e-03,  2.65014080e-02,
         2.79121855e-01, -2.45294305e-01, -2.60821689e-02,
         2.15642706e-02,  3.39433096e-02, -3.64694791e-03,
        -8.70950440e-04,  3.39433096e-02,  5.73391843e-01,
        -5.43966432e-01, -5.14821367e-01, -7.80511862e-01,
        -5.25939983e-01,  3.08028012e-01,  2.94861348e-01,
         1.17053468e+00,  3.26890140e-01, -9.31087254e-01,
         1.61418314e-01,  5.20053383e-01,  2.77595533e-02,
         1.36272024e-02, -1.30976047e-01,  1.09862887e-01,
         9.15181592e-03,  5.57410664e-02, -2.30359339e-01,
         2.04043684e-01,  5.01879525e-03, -1.76310517e-03,
         1.65672213e-01,  1.22684154e-02,  4.45838446e-0

In [30]:
# print(train_features_encoded.columns)
# print(test_features_encoded.columns)
from sklearn.ensemble import RandomForestRegressor

In [31]:
X_train = train_features_encoded
X_test = test_features_encoded
y_train = pd.read_csv('train_labels.csv')['y']

In [32]:
pipeline = make_pipeline(
    StandardScaler(),
    SelectKBest(f_regression),
    Ridge()
)

param_grid = {
    'selectkbest__k': range(1, len(X_train.columns)+1),
    'ridge__alpha': [0.1,1,2,5,10]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3,
                         scoring='neg_mean_absolute_error', verbose=10,
                         refit=True)

gridsearch.fit(X_train,y_train)

Fitting 3 folds for each of 260 candidates, totalling 780 fits
[CV] ridge__alpha=0.1, selectkbest__k=1 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=1, score=-0.1774719030015427, total=   0.1s
[CV] ridge__alpha=0.1, selectkbest__k=1 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=1, score=-0.17921938529820183, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=1 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=1, score=-0.1792340929637908, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=2 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
[Paralle

[CV]  ridge__alpha=0.1, selectkbest__k=2, score=-0.17737002488422668, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=2 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=2, score=-0.17915094837743462, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=2 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=2, score=-0.17910790255247191, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=3 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=3, score=-0.17692861838337862, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=3 ..............................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
[Parallel

[CV]  ridge__alpha=0.1, selectkbest__k=3, score=-0.17845050510984922, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=3 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=3, score=-0.17876866699044547, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=4 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=4, score=-0.17694246422329454, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=4 ..............................
[CV]  ridge__alpha=0.1, selectkbest__k=4, score=-0.17846778950612965, total=   0.0s
[CV] ridge__alpha=0.1, selectkbest__k=4 ..............................


...

[CV]  ridge__alpha=10, selectkbest__k=51, score=-0.16667119976235337, total=   0.0s
[CV] ridge__alpha=10, selectkbest__k=51 ..............................
[CV]  ridge__alpha=10, selectkbest__k=51, score=-0.16883830714686077, total=   0.0s
[CV] ridge__alpha=10, selectkbest__k=52 ..............................
[CV]  ridge__alpha=10, selectkbest__k=52, score=-0.16525706926305617, total=   0.0s
[CV] ridge__alpha=10, selectkbest__k=52 ..............................
[CV]  ridge__alpha=10, selectkbest__k=52, score=-0.16667077994055726, total=   0.0s
[CV] ridge__alpha=10, selectkbest__k=52 ..............................


/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625

[CV]  ridge__alpha=10, selectkbest__k=52, score=-0.16883830470185512, total=   0.0s


/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
[Parallel(n_jobs=1)]: Done 780 out of 780 | elapsed:   45.9s finished
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_regression at 0x1a1b2dbc80>)), ('ridge', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'selectkbest__k': range(1, 53), 'ridge__alpha': [0.1, 1, 2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=10)

In [33]:
print('Best estimator: ',gridsearch.best_estimator_, '\n')
print('Cross-Validation Score: ', gridsearch.best_score_)

Best estimator:  Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=37, score_func=<function f_regression at 0x1a1b2dbc80>)), ('ridge', Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]) 

Cross-Validation Score:  -0.16687613436564935


In [34]:
gridsearch.best_params_

{'ridge__alpha': 0.1, 'selectkbest__k': 37}

In [35]:
selector = gridsearch.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
job_all_services
job_corporate
job_self-employed
job_no_employment
job_student
marital_single
marital_married
education_university.degree
education_unknown
education_basic.9y
education_basic.6y
default_no
default_unknown
housing_yes
housing_no
contact_cellular
contact_telephone
month_may
month_nov
month_jul
month_dec
month_apr
month_mar
month_sep
month_jun
month_oct
day_of_week_thu
day_of_week_mon
poutcome_nonexistent
poutcome_failure
poutcome_success
age
pdays
previous
cons.price.idx
cons.conf.idx
ln_campaign

Features not selected:
job_technician
job_unknown
marital_unknown
education_high.school
education_basic.4y
education_professional.course
education_illiterate
housing_unknown
loan_yes
loan_no
loan_unknown
month_aug
day_of_week_tue
day_of_week_wed
day_of_week_fri


In [36]:
# from sklearn.feature_selection import RFECV

# X_train_scaled = RobustScaler().fit_transform(X_train)
# rfe = RFECV(Ridge(alpha=1.0), scoring='neg_mean_absolute_error', cv=3,verbose=10,refit=True)
# X_train_subset = rfe.fit_transform(X_train_scaled, y_train)

# all_names = X_train.columns
# selected_mask = rfe.support_
# selected_names = all_names[selected_mask]
# unselected_names = all_names[~selected_mask]

# print('Features selected:')
# for name in selected_names:
#     print(name)

# print()
# print('Features not selected:')
# for name in unselected_names:
#     print(name)

In [154]:
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression,f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# X_train_scaled = StandardScaler().fit_transform(X_train)
# rfe = RFECV(DecisionTreeClassifier(random_state=42), scoring='roc_auc', cv=5,verbose=10)
# X_train_subset = rfe.fit_transform(X_train_scaled, y_train)

# all_names = X_train.columns
# selected_mask = rfe.support_
# selected_names = all_names[selected_mask]
# unselected_names = all_names[~selected_mask]

# print('Features selected:')
# for name in selected_names:
#     print(name)

# print()
# print('Features not selected:')
# for name in unselected_names:
#     print(name)

# OKAY FRoM HERE ON IS THE SPLIT T

In [174]:
ohe = ce.OneHotEncoder(use_cat_names=True,handle_unknown='ignore',impute_missing=False)
train_features_encoded = ohe.fit_transform(train_features)
test_features_encoded = ohe.fit_transform(test_features)
train_features_encoded.shape, test_features_encoded.shape

train_features_encoded.drop(columns='default_yes',inplace=True)

X_train = train_features_encoded
X_test = test_features_encoded
y_train = pd.read_csv('train_labels.csv')['y'].values
# y_train = y_train.reshape(-1,1)

In [175]:
# pipeline = make_pipeline(
#     StandardScaler(),
#     SelectKBest(f_classif),
#     RandomForestRegressor()
# )

# param_grid = {
#     'selectkbest__k': range(1,len(X_train.columns)),
#     'RandomForestClassifier__n_estimators': [100],
#     'RandomForestClassifier__criterion': ['gini'],
#     'RandomForestClassifier__max_depth': [10],
# }

# gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3,
#                           scoring='roc_auc', verbose=1, return_train_score=True,
#                           )

# # param_grid ={
# #     'selectkbest__k': range(1,len(X_train.columns)),
# #     'randomforestregressor__n_estimators': [100,150,200],
# #     'randomforestregressor__criterion': ['mse','mae'],
# #     'randomforestregressor__max_depth': [5,7,9,10]
# # }

# # gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error',
# #                           verbose=1)

# gridsearch.fit(X_train,y_train)

In [176]:
# pd.DataFrame(gridsearch.cv_results_)

In [177]:
# gridsearch.best_estimator_

In [178]:
# gridsearch.best_score_

In [179]:
scaler = StandardScaler()
model = RandomForestClassifier(n_jobs=-1)
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

/Users/chrislouie/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [171]:
estimators = np.arange(10,200,10)
scores = []
for n in estimators:
    model.set_params(n_estimators=n)
    model.fit(X_train,y_train)
    scores.append(model.score(X_train,y_train))
    
scores

/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/Users/chrislouie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/Users/chrislouie/anaconda3/lib/

[0.980317891942637,
 0.9877957981289048,
 0.9901589459713185,
 0.9918746560486873,
 0.9923602343724709,
 0.9925868375902367,
 0.9928458126962546,
 0.9928781845845068,
 0.992910556472759,
 0.9929429283610113,
 0.9930400440257681,
 0.9930400440257681,
 0.9930724159140203,
 0.9930724159140203,
 0.9930724159140203,
 0.9930724159140203,
 0.9930724159140203,
 0.9930724159140203,
 0.9930724159140203]

In [191]:
model.set_params(n_estimators=200,max_depth=20,criterion='entropy')
model.fit(X_train,y_train)

submission = sample_submission.copy()
submission['y'] = model.predict_proba(X_test)[:,1:]
submission.head()

,id,y
0,12668,0.108037
1,44327,0.056069
2,26675,0.400163
3,11771,0.109812
4,38173,0.074678


In [192]:
submission.to_csv('submission-014.csv',index=False)

In [38]:
X_train_subset = pd.DataFrame(X_train_subset,columns=selected_names)

In [39]:
print(X_train.shape,X_train_subset.shape)

(30891, 52) (30891, 36)


In [57]:
X_train_subset.head()

,job_all_services,job_corporate,job_self-employed,job_technician,job_no_employment,job_student,marital_single,marital_married,education_university.degree,education_unknown,...,day_of_week_wed,poutcome_nonexistent,poutcome_failure,poutcome_success,age,pdays,previous,cons.price.idx,cons.conf.idx,ln_campaign
0,2.674876,-1.103155,-0.270280,-0.443116,-0.266955,-0.148441,1.242130,-1.236922,-0.647280,-0.210859,...,-0.496143,0.400663,-0.341481,-0.187303,-1.058708,0.197878,-0.351159,-1.178535,-1.229930,2.394382
1,-0.373849,0.906491,-0.270280,-0.443116,-0.266955,-0.148441,-0.805069,0.808459,1.544927,-0.210859,...,-0.496143,-2.495866,2.928419,-0.187303,1.344914,0.197878,1.648042,-0.648114,-0.322731,-0.940004
2,-0.373849,-1.103155,3.699867,-0.443116,-0.266955,-0.148441,-0.805069,0.808459,-0.647280,4.742495,...,-0.496143,0.400663,-0.341481,-0.187303,1.441059,0.197878,-0.351159,0.592414,-0.473931,0.650903
3,-0.373849,0.906491,-0.270280,-0.443116,-0.266955,-0.148441,-0.805069,0.808459,-0.647280,4.742495,...,-0.496143,0.400663,-0.341481,-0.187303,0.287320,0.197878,-0.351159,-1.178535,-1.229930,1.654653
4,-0.373849,0.906491,-0.270280,-0.443116,-0.266955,-0.148441,-0.805069,0.808459,-0.647280,-0.210859,...,2.015549,0.400663,-0.341481,-0.187303,1.344914,0.197878,-0.351159,0.723723,0.886867,1.390632


In [58]:
X_test_subset = rfe.transform(test_features_encoded)

In [59]:
print(test_features_encoded.shape, X_test_subset.shape)

(10297, 52) (10297, 36)


In [65]:
from sklearn.linear_model import LinearRegression
# pipeline = make_pipeline(
#     LinearRegression()
# )

# pipeline.fit(X_train_subset, y_train)

submission = sample_submission.copy()
submission['y'] = gridsearch.predict_log_proba(X_test_subset)[:,1:]

AttributeError: 'Ridge' object has no attribute 'predict_log_proba'

In [55]:
submission.head()

,id,y
0,12668,-55.144575
1,44327,-55.055598
2,26675,-54.474324
3,11771,-55.084597
4,38173,-55.057281


In [ ]:
submission.to_csv('submission-009.csv',index=False)